# Análisis de Libros SQL - "BookApp"

## 1. Objetivos de estudio:
**Generar insights estratégicos para el desarrollo de un nuevo producto basado en datos de libros, autores, editoriales, ratings y opiniones de usuarios:**
- Encontrar el número de libros publicados después del 1 de enero de 2000.
- Encontrar el número de reseñas de usuarios y la calificación promedio para cada libro.
- Identificar la editorial que ha publicado el mayor número de libros con más de 50 páginas (excluir folletos y publicaciones similares).
- Identificar al autor que tiene la más alta calificación promedio del libro (solo libros con al menos 50 calificaciones).
- Encontrar el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.

### 1.1 Importar librerías y realizar conexión a base de datos.

In [1]:
# Importar librerías
import pandas as pd
import os
from sqlalchemy import create_engine
from dotenv import load_dotenv

In [2]:
# Conexión con base de datos

# Cargar variables de entorno desde .env
load_dotenv()

db_config = {
    'user': os.getenv('DB_USER'),
    'pwd': os.getenv('DB_PASSWORD'),
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT'),
    'db': os.getenv('DB_NAME')
}

connection_string = f"postgresql://{db_config['user']}:{db_config['pwd']}@{db_config['host']}:{db_config['port']}/{db_config['db']}"

engine = create_engine(
    connection_string,
    connect_args={'sslmode': 'verify-full', 'sslrootcert': '../data/raw/CA.pem'}
)

## 2. Estudiar las tablas (primeras filas):

In [3]:
# Explorar estructura de cada tabla
tables = ['books', 'authors', 'publishers', 'ratings', 'reviews']

for table in tables:
    print(f"Primeras filas de '{table}':\n")
    query = f"SELECT * FROM {table} LIMIT 10;"
    df = pd.io.sql.read_sql(query, con=engine)
    display(df)
    
    count_query = f"SELECT COUNT(*) FROM {table};"
    count_df = pd.io.sql.read_sql(count_query, con=engine)
    print(f"Total de filas en '{table}': {count_df.iloc[0,0]}")
    print("="*120,"\n")

Primeras filas de 'books':



,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
5,6,257,1st to Die (Women's Murder Club #1),424,2005-05-20,116
6,7,258,2nd Chance (Women's Murder Club #2),400,2005-05-20,116
7,8,260,4th of July (Women's Murder Club #4),448,2006-06-01,318
8,9,563,A Beautiful Mind,461,2002-02-04,104
9,10,445,A Bend in the Road,341,2005-04-01,116


Total de filas en 'books': 1000

Primeras filas de 'authors':



,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
5,6,Alan Paton
6,7,Albert Camus/Justin O'Brien
7,8,Aldous Huxley
8,9,Aldous Huxley/Christopher Hitchens
9,10,Aleksandr Solzhenitsyn/H.T. Willetts


Total de filas en 'authors': 636

Primeras filas de 'publishers':



...

Total de filas en 'publishers': 340

Primeras filas de 'ratings':



,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
5,6,3,johnsonamanda,4
6,7,3,scotttamara,5
7,8,3,lesliegibbs,5
8,9,4,abbottjames,5
9,10,4,valenciaanne,4


Total de filas en 'ratings': 6456

Primeras filas de 'reviews':



,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
5,6,3,lesliegibbs,Analysis no several cause international.
6,7,4,valenciaanne,One there cost another. Say type save. With pe...
7,8,4,abbottjames,Within enough mother. There at system full rec...
8,9,5,npowers,Thank now focus realize economy focus fly. Ite...
9,10,5,staylor,Game push lot reduce where remember. Including...


Total de filas en 'reviews': 2793



## 3. Consulta SQL para cada una de las tareas:
### 3.1 Encontrar el número de libros publicados después del 1 de enero de 2000.

In [4]:
# Encontrar el número de libros publicados después del 1 de enero de 2000
query_ex1 = """
SELECT COUNT(*) as total_books
FROM books
WHERE publication_date > '2000-01-01';
"""

result_ex1 = pd.io.sql.read_sql(query_ex1, con=engine)
print("Total de libros publicados después del 1 de enero de 2000:\n")
display(result_ex1)

Total de libros publicados después del 1 de enero de 2000:



,total_books
0,819


### 3.2 Encontrar el número de reseñas de usuarios y la calificación promedio para cada libro.

In [5]:
# Encontrar número de reseñas y calificación promedio por libro
query_ex2 = """
SELECT 
    b.book_id,
    b.title,
    COUNT(DISTINCT r.review_id) as review_count,
    COUNT(DISTINCT rt.rating_id) as rating_count,
    ROUND(AVG(rt.rating), 2) as avg_rating
FROM books b
LEFT JOIN reviews r ON b.book_id = r.book_id
LEFT JOIN ratings rt ON b.book_id = rt.book_id
GROUP BY b.book_id, b.title
ORDER BY review_count DESC, avg_rating DESC;
"""

result_ex2 = pd.io.sql.read_sql(query_ex2, con=engine)
print("Libros por número de reseñas y calificación promedio:\n")
display(result_ex2)

Libros por número de reseñas y calificación promedio:



,book_id,title,review_count,rating_count,avg_rating
0,948,Twilight (Twilight #1),7,160,3.66
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,82,4.41
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,80,4.29
3,656,The Book Thief,6,53,4.26
4,734,The Glass Castle,6,29,4.21
...,...,...,...,...,...
995,191,Disney's Beauty and the Beast (A Little Golden...,0,1,4.00
996,387,Leonardo's Notebooks,0,2,4.00
997,221,Essential Tales and Poems,0,3,4.00
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3,3.67


### 3.3 Identificar la editorial que ha publicado el mayor número de libros con más de 50 páginas (excluyendo folletos y publicaciones similares).

In [6]:
# Encontrar editorial con mayor número de libros de más de 50 páginas 
query_ex3 = """
SELECT 
    p.publisher_id,
    p.publisher,
    COUNT(b.book_id) as book_count,
    ROUND(AVG(b.num_pages), 2) as avg_pages,
    MIN(b.num_pages) as min_pages,
    MAX(b.num_pages) as max_pages
FROM publishers p
JOIN books b ON p.publisher_id = b.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher_id, p.publisher
ORDER BY book_count DESC;
"""

result_ex3 = pd.io.sql.read_sql(query_ex3, con=engine)
print("Top editoriales por número de libros con más de 50 páginas:\n")
display(result_ex3)

Top editoriales por número de libros con más de 50 páginas:



,publisher_id,publisher,book_count,avg_pages,min_pages,max_pages
0,212,Penguin Books,42,380.02,80,1023
1,309,Vintage,31,351.45,120,647
2,116,Grand Central Publishing,25,425.24,204,608
3,217,Penguin Classics,24,510.29,70,1556
4,33,Ballantine Books,19,499.74,272,1728
...,...,...,...,...,...,...
329,34,Ballantine Books (NY),1,422.00,422,422
330,225,Plaza y Janés,1,432.00,432,432
331,138,HarperCollinsPublishers,1,184.00,184,184
332,245,Random House: Modern Library,1,272.00,272,272


### 3.4 Identificar al autor que tiene la más alta calificación promedio del libro (solo libros con al menos 50 calificaciones).

In [7]:
# Encontrar autor que tiene la más alta calificación promedio de libro (solo libros con al menos 50 calificaciones)
query_ex4 = """
SELECT 
    a.author_id,
    a.author,
    b.book_id,
    b.title,
    COUNT(rt.rating_id) as rating_count,
    ROUND(AVG(rt.rating), 2) as avg_rating
FROM authors a
JOIN books b ON a.author_id = b.author_id
JOIN ratings rt ON b.book_id = rt.book_id
GROUP BY a.author_id, a.author, b.book_id, b.title
HAVING COUNT(rt.rating_id) >= 50
ORDER BY avg_rating DESC;
"""

result_ex4 = pd.io.sql.read_sql(query_ex4, con=engine)
print("Top autores con mejor calificación promedio (mínimo 50 calificaciones):\n")
display(result_ex4)

Top autores con mejor calificación promedio (mínimo 50 calificaciones):



,author_id,author,book_id,title,rating_count,avg_rating
0,236,J.K. Rowling/Mary GrandPré,302,Harry Potter and the Prisoner of Azkaban (Harr...,82,4.41
1,240,J.R.R. Tolkien,722,The Fellowship of the Ring (The Lord of the Ri...,74,4.39
2,236,J.K. Rowling/Mary GrandPré,299,Harry Potter and the Chamber of Secrets (Harry...,80,4.29
3,402,Markus Zusak/Cao Xuân Việt Khương,656,The Book Thief,53,4.26
4,236,J.K. Rowling/Mary GrandPré,300,Harry Potter and the Half-Blood Prince (Harry ...,73,4.25
5,236,J.K. Rowling/Mary GrandPré,301,Harry Potter and the Order of the Phoenix (Har...,75,4.19
6,376,Louisa May Alcott,399,Little Women,52,4.19
7,240,J.R.R. Tolkien,750,The Hobbit or There and Back Again,88,4.13
8,498,Rick Riordan,779,The Lightning Thief (Percy Jackson and the Oly...,62,4.08
9,621,William Golding,405,Lord of the Flies,71,3.90


### 3.5 Encontrar el número promedio de reseñas de texto entre usuarios que calificaron más de 50 libros.

In [8]:
# Encontrar número promedio de reseñas de texto entre usuarios que calificaron más de 50 libros
query_ex5 = """
SELECT 
    ROUND(AVG(user_review_count), 2) as avg_reviews_per_user
FROM (
    SELECT 
        r.username,
        COUNT(DISTINCT r.review_id) as user_review_count
    FROM reviews r
    WHERE r.username IN (
        SELECT username
        FROM ratings
        GROUP BY username
        HAVING COUNT(DISTINCT book_id) > 50
    )
    GROUP BY r.username
) as active_users_reviews;
"""

result_ex5 = pd.io.sql.read_sql(query_ex5, con=engine)
print("Promedio de reseñas de texto entre usuarios que calificaron más de 50 libros:\n")
display(result_ex5)

Promedio de reseñas de texto entre usuarios que calificaron más de 50 libros:



,avg_reviews_per_user
0,24.33


In [9]:
# Usuarios que calificaron más de 50 libros
query_verification_ex5 = """
SELECT COUNT(*) as total_users
FROM (
    SELECT username
    FROM ratings
    GROUP BY username
    HAVING COUNT(DISTINCT book_id) > 50
) as active_users;
"""

verification_result_ex5 = pd.io.sql.read_sql(query_verification_ex5, con=engine)
print(f"Número de usuarios que calificaron más de 50 libros: {verification_result_ex5.iloc[0,0]}")

Número de usuarios que calificaron más de 50 libros: 6


## 4. Generar resultados de cada consulta:

1. Libros publicados después del 1 de enero de 2000:
    - 819 libros.
    <br>

2. Número de reseñas y calificación promedio por libro:
    - Libro con más reseñas: "Twilight(Twilight #1)" con 7 reseñas y 160 calificaciones.
    - Mejor calificados: Libros de Harry Potter ("...and the Prisoner of Azkaban" y "and the Chamber of Secrets") con ratings superiores a 4.25.
    - Total de libros analizados: 1000 libros.
    <br>

3. Editorial líder en libros extensos (más de 50 páginas):
    - Editorial líder: "Penguin Books", con 42 libros.
    <br>

4. Autor con mejor calificación promedio (de libros con al menos 50 caliicaciones):
    - Autor/a: J.K. Rowling.
    - Libro: "Harry Potter and the Prisoner of Azkaban".
    - Calificación: 4.41/5 (con 82 calificaciones).
    <br>

5. Reseñas de usuarios que calificaron más de 50 libros:
    - Promedio de reseñas por usuario: 24.33 reseñas.
    - Usuarios que calificaron más de 50 libros: 6 usuarios.

## 5. Describir conclusiones para cada una de las tareas:

1. Libros publicados después del 1 de enero de 2000:
    - En el mercado predominan publicaciones recientes, siendo un 81.90% de las publicaciones después del año 2000. Sugiere que los usuarios prefieren contenido contemporaneo. Sería favorable para una nueva app enfocada en tendencias actuales, incluyendo principalmente géneros de fantasía, suspenso y drama.
    <br>

2. Número de reseñas y calificación promedio por libro:
    - Existe una correlación positiva entre el número de reseñas (popularidad) y la calificación promedio (calidad). Los libros de "Harry Potter" demuestran que es posible mantener alta calidad con alto engagement; mientras que "Twilight" lidera las interacciones (7 reseñas y 160 calificaciones).
    -  Los libros con 0 reseñas pero que tienen calificaciones presentan una oportunidad para incentivar reseñas.
    <br>

3. Editorial líder en libros extensos (más de 50 páginas):
    - "Penguin Books" es la editorial más prolífica en contenido de valor (42 libros > 50 páginas), seguida por "Vintage" (31) y "Grand Central Publishing" (25). Estas editoriales representan socios potenciales prioritarios para garantizar un catálogo de calidad en el nuevo producto.
    <br>

4. Autor con mejor calificación promedio (de libros con al menos 50 caliicaciones):
    - J.K. Rowling es la autora con mejor recepción en el mercado. Siendo "Harry Potter and the Prisoner of Azkaban" el libro con la calificación más alta (4.41/5). La consistencia de la serie de "Harry Potter# en el top, demuestra el valor de las sagas bien desarrolladas.
    <br>

5. Reseñas de usuarios que calificaron más de 50 libros:
    - Se identifica un núcleo pequeño pero crucial de 6 usuarios que calificaron más de 50 libros, generando un promedio de 24.33 reseñas por cada uno. Estos usuarios son clave para la vitalidad de la comunidad y el engagement, por lo que se recomienda otorgarles atención especial en estrategias específicas de retención y reconocimiento.